In [1]:
import pandas as pd
import os
import re

In [10]:
def find_besteye(df_msg, default='R'):
    val_msgs = (df_msg[df_msg['text'].str.contains('CAL VALIDATION')][-2:]).to_numpy(dtype=str)
    if not len(val_msgs) or 'ABORTED' in val_msgs[0][1]:
        return default

    left_index = int('LEFT' in val_msgs[1][1])
    right_index = 1 - left_index
    lefterror_index, righterror_index = val_msgs[left_index][1].split().index('ERROR'), val_msgs[right_index][
        1].split().index('ERROR')
    left_error = float(val_msgs[left_index][1].split()[lefterror_index + 1])
    right_error = float(val_msgs[right_index][1].split()[righterror_index + 1])

    return 'L' if left_error < right_error else 'R'


def filter_msgs(df_msg, cutout='validation'):
    first_index = df_msg.index[df_msg['text'].str.contains(cutout)].tolist()[0]

    return df_msg[first_index:]


def is_binocular(df_fix):
    return len(df_fix['eye'].unique()) > 1


def keep_besteye(df_fix, df_msg, default='R'):
    best_eye = default
    if is_binocular(df_fix):
        best_eye = find_besteye(df_msg, default)
        df_fix = df_fix[df_fix['eye'] == best_eye]

    return df_fix, best_eye


def extract_calpoints(df_msg, best_eye, legend='Calibration points', npoints=9):
    calpoints_msg = df_msg[df_msg['text'].str.contains(legend)]
    calpoints = []
    if not calpoints_msg.empty:
        if len(calpoints_msg) >= 2:
            calpoints_msgidx = calpoints_msg.iloc[-2].name if best_eye == 'L' else calpoints_msg.iloc[-1].name
        else:
            calpoints_msgidx = calpoints_msg.iloc[0].name
        calpoints = df_msg.loc[calpoints_msgidx + 1:calpoints_msgidx + npoints]['text'].to_numpy()
        calpoints = [list(map(lambda x: float(x.replace(',', '')), msg.split()[1:3])) for msg in calpoints]
    calpoints = pd.DataFrame(calpoints, columns=['x', 'y'])

    return calpoints


def extract_valpoints(df_msg, best_eye, legend='VALIDATE', npoints=9):
    valpoints_msg = df_msg[df_msg['text'].str.contains(legend)]
    if len(valpoints_msg) > npoints:
        besteye_legend = 'RIGHT' if best_eye == 'R' else 'LEFT'
        valpoints_msg = valpoints_msg[valpoints_msg['text'].str.contains(besteye_legend)]
    valpoints_msg = valpoints_msg['text'].to_numpy()
    points = [msg.split('at')[1].split('OFFSET')[0].split(',') for msg in valpoints_msg]
    regexp = r'(-?\d+\.\d+)\s*,\s*(-?\d+\.\d+)\s*pix'
    offsets = [re.findall(regexp, msg.split('at')[1].split('OFFSET')[1]) for msg in valpoints_msg]
    offsets = [(float(offset[0][0]), float(offset[0][1])) for offset in offsets]
    points = [(int(point[0]), int(point[1])) for point in points]
    valpoints = pd.DataFrame(points, columns=['x', 'y']).astype(int)
    valoffsets = pd.DataFrame(offsets, columns=['x', 'y']).astype(float)

    return valpoints, valoffsets

In [2]:
from pyxations.pyxations.bids_formatting import dataset_to_bids, compute_derivatives_for_dataset

In [ ]:
# Example usage:
current_folder = os.getcwd()
files_folder_path = os.path.join(current_folder,"example_files")
bids_dataset_folder = dataset_to_bids(current_folder,files_folder_path,"example_dataset")
msg_keywords = ["begin","end","press"]
compute_derivatives_for_dataset(bids_dataset_folder, msg_keywords)

